In [30]:
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense

import sklearn
import pandas
import numpy
import json
import os
from sklearn.model_selection import train_test_split

In [45]:
image_width = 150
image_height = 150
image_channels = 3
batch_size = 15
epochs = 1

In [32]:
model = tensorflow.keras.Sequential()

In [33]:
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (image_width, image_height, image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [34]:
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [35]:
model.add(Conv2D(150, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [36]:
model.add(Flatten())

In [37]:
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [38]:
model.add(Dense(2, activation = 'softmax'))

In [39]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)       

In [56]:
types = ['jpg', 'png']
categories = []

cats = os.listdir("./data/train/cats")
dogs = os.listdir("./data/train/dogs")

cats = [cat for cat in cats if len(cat.split(".")) > 2 if cat.split(".")[2] in types]
dogs = [dog for dog in dogs if len(dog.split(".")) > 2 if dog.split(".")[2] in types]

for i in range(0, len(cats)):
    categories.append(0)

for i in range(0, len(dogs)):
    categories.append(1)
        
dataframe = pandas.DataFrame({
    'filename' : cats + dogs,
    'category' : categories
})

dataframe["category"] = dataframe["category"].replace({0: 'cat', 1: 'dog'})


In [57]:
class CustomCallback(tensorflow.keras.callbacks.Callback):
    
    def on_train_begin(self):
        print("Starting training on this.date")


In [62]:
train_df, validate_df = train_test_split(dataframe, test_size = 0.20, random_state = 42)
train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [65]:
#train_data_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
#    rotation_range = 15, 
#    rescale = 1. / 255, 
#    shear_range = 0.1, 
#    zoom_range = 0.2, 
#    horizontal_flip = True, 
#    width_shift_range = 0.1, 
#    height_shift_range = 0.1)

#train_generator = train_data_generator.flow_from_dataframe(train_df, "./data/train/", x_col = 'filename', y_col = 'category', target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)


#validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255)
#validation_generator = validation_datagen.flow_from_dataframe(validate_df, "./data/validation/",  x_col='filename', y_col='category', target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)

In [66]:
model.fit(dataframe, epochs = epochs, validation_steps=800,steps_per_epoch=10)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (8000, 2)